# Projeto Semantix

COLUNAS:

- Date: Data
- Time: Horário
- Comp: "Premier League"
- Round: Rodada da partida
- Day: Dia da Semana
- Venue: "Home"
- Result: Resultado da partida ("W", "L" ou "D")
- GF: Gols Marcados
- GA: Gols Sofridos
- Opponent: Oponente
- xG: Gols Marcados esperados
- xGA: Gols Sofridos esperados
- Poss: Posse de bola
- Attendance: Público presente
- Captain: Capitão do Time
- Formation: Formação do Time
- Referee: Árbitro
- Sh: Total de Chutes
- SoT: Total de Chutes a Gol
- Dist: Distância Média  dos Chutes
- FK: Chutes de Falta
- PK: Penaltis Convertidos
- PKatt: Penaltis Cobrados
- Season: "2024"
- Team: Time Mandante
- Captain_Opponent: Capitão do Time
- Formation_Opponent: Formação do Time
- Sh_Opponent: Total de Chutes
- SoT_Opponent: Total de Chutes a Gol
- Dist_Opponent: Distância Média dos Chutes
- FK_Opponent: Chutes de Falta
- PK_Opponent: Penaltis Convertidos
- PKatt_Opponent: Penaltis Cobrados

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import streamlit
from script import analise
from script import get_dict
from script import Legenda

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

sns.set_theme()

%reload_ext watermark
%watermark -iv -v

Python implementation: CPython
Python version       : 3.12.2
IPython version      : 8.22.2

pandas    : 2.1.4
matplotlib: 3.7.5
streamlit : 1.32.2
seaborn   : 0.13.2
numpy     : 1.26.4



In [3]:
Teams =          {'Arsenal'                : 1,
                  'AstonVilla'             : 2,
                  'Bournemouth'            : 3,
                  'Brentford'              : 4,
                  'BrightonandHoveAlbion'  : 5,
                  'Burnley'                : 6,
                  'Chelsea'                : 7,
                  'CrystalPalace'          : 8,
                  'Everton'                : 9,
                  'Fulham'                 : 10,
                  'Liverpool'              : 11,
                  'LutonTown'              : 12,
                  'ManchesterCity'         : 13,
                  'ManchesterUnited'       : 14,
                  'NewcastleUnited'        : 15,
                  'NottinghamForest'       : 16,
                  'SheffieldUnited'        : 17,
                  'TottenhamHotspur'       : 18,
                  'WestHamUnited'          : 19,
                  'WolverhamptonWanderers' : 20}

Teams_Opponent = {'Arsenal'         : 1, 
                  'Aston Villa'     : 2,     
                  'Bournemouth'     : 3,     
                  'Brentford'       : 4,   
                  'Brighton'        : 5,  
                  'Burnley'         : 6, 
                  'Chelsea'         : 7, 
                  'Crystal Palace'  : 8,        
                  'Everton'         : 9, 
                  'Fulham'          : 10, 
                  'Liverpool'       : 11,   
                  'Luton Town'      : 12,    
                  'Manchester City' : 13,         
                  'Manchester Utd'  : 14,        
                  'Newcastle Utd'   : 15,       
                  "Nott'ham Forest" : 16,         
                  'Sheffield Utd'   : 17, 
                  'Tottenham'       : 18,
                  'West Ham'        : 19,
                  'Wolves'          : 20}

Matchweek =      {'Matchweek 1': 1,
                  'Matchweek 2': 2,
                  'Matchweek 3': 3,
                  'Matchweek 4': 4,
                  'Matchweek 5': 5,
                  'Matchweek 6': 6,
                  'Matchweek 7': 7,
                  'Matchweek 8': 8,
                  'Matchweek 9': 9,
                  'Matchweek 10': 10,
                  'Matchweek 11': 11,
                  'Matchweek 12': 12,
                  'Matchweek 13': 13,
                  'Matchweek 14': 14,
                  'Matchweek 15': 15,
                  'Matchweek 16': 16,
                  'Matchweek 17': 17,
                  'Matchweek 18': 18,
                  'Matchweek 19': 19,
                  'Matchweek 20': 20,
                  'Matchweek 21': 21,
                  'Matchweek 22': 22,
                  'Matchweek 23': 23,
                  'Matchweek 24': 24}

weekday =        {'Mon': 0, 'Tue': 1, 'Wed': 2, 'Thu':3, 'Fri': 4, 'Sat': 5, 'Sun': 6}

In [6]:
# Lendo CSV.
df = pd.read_csv('./input/matches.csv', parse_dates=['Date'])

# Drop nas colunas sem relevância.
df.drop(columns=['Unnamed: 0', 'Notes', 'Match Report', 'Comp'], inplace=True)

# Transformando os times, rodada, dia da semana, jogo casa/fora e 'Result' em códigos numéricos.
df['Team_cod'] = df['Team'].map(Teams)
df['Opponent_cod'] = df['Opponent'].map(Teams_Opponent)
df['Opponent'] = df['Opponent_cod'].map({valor: chave for chave, valor in Teams.items()})
df['Round'] = df['Round'].map(Matchweek)
df['Day'] = df['Day'].map(weekday)
df['Venue'] = df['Venue'].map({'Away': 0, 'Home': 1})
df['Points'] = df['Result'].apply(lambda x: 3 if x == 'W' else (1 if x == 'D' else 0))

# Amostra dos dados.
df.head()

,Date,Time,Round,Day,Venue,Result,GF,GA,Opponent,xG,...,SoT,Dist,FK,PK,PKatt,Season,Team,Team_cod,Opponent_cod,Points
0,2023-08-13,16:30,1,6,0,D,1.0,1.0,Chelsea,1.3,...,1.0,17.8,0.0,0,0,2024,Liverpool,11,7,1
1,2023-08-19,15:00,2,5,1,W,3.0,1.0,Bournemouth,3.0,...,9.0,16.8,1.0,0,1,2024,Liverpool,11,3,3
2,2023-08-27,16:30,3,6,0,W,2.0,1.0,NewcastleUnited,0.9,...,4.0,17.2,1.0,0,0,2024,Liverpool,11,15,3
3,2023-09-03,14:00,4,6,1,W,3.0,0.0,AstonVilla,2.5,...,4.0,14.7,0.0,0,0,2024,Liverpool,11,2,3
4,2023-09-16,12:30,5,5,0,W,3.0,1.0,WolverhamptonWanderers,2.5,...,5.0,15.8,0.0,0,0,2024,Liverpool,11,20,3


In [4]:
analise(df, "Points")

,dtype,contagem,missing,nunique,papel
Date,datetime64[ns],476,0,75,covariavel
Time,object,476,0,11,covariavel
Round,int64,476,0,24,covariavel
Day,int64,476,0,7,covariavel
Venue,int64,476,0,2,covariavel
Result,object,476,0,3,covariavel
GF,float64,476,0,8,covariavel
GA,float64,476,0,8,covariavel
Opponent,object,476,0,20,covariavel
xG,float64,476,0,43,covariavel


In [5]:
for i in df['Team'].unique():
    for j in df['Opponent'].unique():

        Template = i + ' x ' + j

        df.loc[((df['Team'] == i) & (df["Opponent"] == j)) | ((df['Team'] == j) & (df["Opponent"] == i)), 'Matches'] = Template
        
df.head()

,Date,Time,Round,Day,Venue,Result,GF,GA,Opponent,xG,...,Dist,FK,PK,PKatt,Season,Team,Team_cod,Opponent_cod,Points,Matches
0,2023-08-13,16:30,1,6,0,D,1.0,1.0,Chelsea,1.3,...,17.8,0.0,0,0,2024,Liverpool,11,7,1,Chelsea x Liverpool
1,2023-08-19,15:00,2,5,1,W,3.0,1.0,Bournemouth,3.0,...,16.8,1.0,0,1,2024,Liverpool,11,3,3,Bournemouth x Liverpool
2,2023-08-27,16:30,3,6,0,W,2.0,1.0,NewcastleUnited,0.9,...,17.2,1.0,0,0,2024,Liverpool,11,15,3,NewcastleUnited x Liverpool
3,2023-09-03,14:00,4,6,1,W,3.0,0.0,AstonVilla,2.5,...,14.7,0.0,0,0,2024,Liverpool,11,2,3,AstonVilla x Liverpool
4,2023-09-16,12:30,5,5,0,W,3.0,1.0,WolverhamptonWanderers,2.5,...,15.8,0.0,0,0,2024,Liverpool,11,20,3,WolverhamptonWanderers x Liverpool


In [6]:
df_home = df[df['Venue'] == 1].copy().reset_index(drop=True)
df_away = df[df['Venue'] == 0].copy().reset_index(drop=True)

In [59]:
drop_columns = ["Date",
                "Time",
                "Matches",
                "Date_Opponent",
                "Time_Opponent",
                "Day_Opponent",
                "Venue_Opponent",
                "Result_Opponent",
                "Captain",
                "Captain_Opponent",
                "Opponent",
                "GF",
                "GA",
                'PK',
                'PKatt',
                "Points",
                "Season",
                "Venue",
                "Referee",
                "GF_Opponent",
                "GA_Opponent",
                "Opponent_Opponent",
                "xG_Opponent",
                "xGA_Opponent",
                "Referee_Opponent",
                "Season_Opponent",
                "Team_Opponent",
                "Team",
                "Team_cod_Opponent",
                "Opponent_cod_Opponent",
                'PK_Opponent',
                'PKatt_Opponent',
                'Points_Opponent']

merged_df = pd.merge(df_home, df_away, on=['Matches', 'Round'], suffixes=['', '_Opponent'])
merged_df.drop(columns=drop_columns, inplace=True)
merged_df

,Round,Day,Result,xG,xGA,Poss,Attendance,Formation,Sh,SoT,...,FK,Team_cod,Opponent_cod,Poss_Opponent,Attendance_Opponent,Formation_Opponent,Sh_Opponent,SoT_Opponent,Dist_Opponent,FK_Opponent
0,2,5,W,3.0,1.3,64.0,53145.0,4-3-3,25.0,9.0,...,1.0,11,3,36.0,53145.0,4-2-3-1,13.0,5.0,19.7,1.0
1,4,6,W,2.5,0.7,63.0,50109.0,4-3-3,17.0,4.0,...,0.0,11,2,37.0,50109.0,4-4-2,9.0,3.0,12.5,0.0
2,6,6,W,3.0,1.1,63.0,50136.0,4-3-3,21.0,6.0,...,2.0,11,19,37.0,50136.0,4-2-3-1,11.0,4.0,13.5,0.0
3,9,5,W,2.2,0.1,77.0,50201.0,4-3-3,25.0,5.0,...,1.0,11,9,23.0,50201.0,4-4-1-1,6.0,1.0,35.2,0.0
4,10,6,W,3.2,0.4,72.0,50143.0,4-3-3,21.0,8.0,...,2.0,11,16,28.0,50143.0,5-4-1,9.0,1.0,15.6,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233,15,2,L,1.0,1.7,25.0,31406.0,3-4-3,6.0,1.0,...,1.0,17,11,75.0,31406.0,4-3-3,15.0,8.0,16.4,0.0
234,16,5,W,0.8,0.6,41.0,28509.0,4-1-4-1,9.0,4.0,...,0.0,17,4,59.0,28509.0,4-3-3,10.0,4.0,16.6,0.0
235,19,1,L,2.4,0.6,51.0,31041.0,4-4-2,21.0,4.0,...,1.0,17,12,49.0,31041.0,3-4-3,12.0,4.0,17.9,1.0
236,21,6,D,2.5,2.3,57.0,29164.0,4-1-4-1,20.0,5.0,...,1.0,17,19,43.0,29164.0,4-4-1-1,15.0,4.0,14.9,0.0


In [60]:
analise(merged_df, "Result")

,dtype,contagem,missing,nunique,papel
Round,int64,238,0,24,covariavel
Day,int64,238,0,7,covariavel
Result,object,238,0,3,resposta
xG,float64,238,0,37,covariavel
xGA,float64,238,0,37,covariavel
Poss,float64,238,0,57,covariavel
Attendance,float64,238,0,230,covariavel
Formation,object,238,0,15,covariavel
Sh,float64,238,0,30,covariavel
SoT,float64,238,0,14,covariavel


In [61]:
df_dummy = pd.get_dummies(merged_df.drop(columns='Result'), drop_first=True, dtype='int')

In [62]:
X = df_dummy
y = merged_df['Result']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=123)

In [63]:
clf = RandomForestClassifier(n_estimators=100, random_state=123)

clf.fit(X=X_train, y=y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)

print(f'Acurácia do modelo Random Forest: {accuracy:.2%}')

Acurácia do modelo Random Forest: 61.11%


Alguns insights que podem ser obtidos a partir deste dataset são:

1. O time Liverpool teve um desempenho consistente em casa, registrando principalmente vitórias ou empates nos jogos.

2. O time Sheffield United teve um desempenho mais variado, com vitórias, derrotas e empates em casa.

3. O número médio de chutes a gol (Sh) do Liverpool em casa é de aproximadamente 20, enquanto o número médio do Sheffield United é de aproximadamente 12.

4. O Liverpool teve uma média de gols marcados (GF) em casa de cerca de 2,5 por jogo, enquanto o Sheffield United teve uma média de cerca de 1 gol por jogo.

5. A equipe enfrentou adversários como Bournemouth, Aston Villa, West Ham, Everton, além de outros ao longo das semanas de competição na Premier League.

Esses são apenas alguns insights iniciais que podem ser extraídos. Mais análises detalhadas podem fornecer informações adicionais sobre o desempenho das equipes.

Este dataset contém informações sobre partidas da Premier League da temporada 2023/2024, incluindo a data, horário, local, resultado, número de gols marcados e sofridos, adversário, estatísticas como posse de bola, chutes, chutes no alvo, entre outros. Também apresenta detalhes sobre o árbitro, capitão do time, formação tática, entre outras informações relevantes sobre cada jogo.

Há um total de 238 registros no dataset, cada um representando uma partida de um time específico da Premier League. Os times incluídos são o Liverpool e o Sheffield United.

Esses dados podem ser utilizados para análises estatísticas, avaliação de desempenho das equipes, predições de resultados, entre outros fins relacionados ao futebol e à Premier League.

Para realizar uma análise detalhada com a variável resposta 'Result', podemos começar analisando a frequência de vitórias, empates e derrotas das equipes ao longo da temporada. Podemos também verificar se existe alguma relação entre o resultado de uma partida e variáveis como posse de bola, número de chutes, distância média dos chutes, entre outras.

Além disso, podemos criar gráficos para visualizar essas relações de forma mais clara. Por exemplo, um gráfico de barras mostrando a distribuição de vitórias, empates e derrotas ao longo da temporada, um gráfico de dispersão mostrando a relação entre posse de bola e resultado da partida, um gráfico de boxplot mostrando a distribuição dos chutes e sua relação com o resultado, entre outros.

Com base nesses gráficos e análises, podemos identificar padrões e possíveis insights sobre quais variáveis influenciam mais nos resultados das partidas e como as equipes podem melhorar seu desempenho.

Para realizar uma análise detalhada dos resultados das partidas com base na variável 'Result', podemos considerar as seguintes variáveis como as mais influentes:

1. xG (Expected Goals): A expectativa de gols pode influenciar diretamente nos resultados das partidas, uma vez que equipes com um valor alto de xG tendem a marcar mais gols e, consequentemente, têm maiores chances de vencer.

2. Possession (Possessão de bola): A posse de bola durante o jogo também pode ser um fator determinante para o resultado final. Equipes com maior posse de bola geralmente têm mais controle do jogo e mais oportunidades de marcar gols.

3. Sh (Shots): O número total de chutes ao gol pode indicar a efetividade do ataque de uma equipe. Equipes com um alto número de chutes tendem a criar mais oportunidades de gol e, portanto, têm mais chances de vencer.

4. GA (Goals Against): A quantidade de gols sofridos também é um fator crucial. Equipes com uma defesa sólida e que sofrem poucos gols têm maiores chances de obter resultados positivos.

Além disso, outras variáveis como a formação tática, o desempenho do capitão da equipe, o número de faltas cometidas (FK) e a precisão nos chutes a gol (SoT) também podem influenciar nos resultados das partidas. Uma análise mais aprofundada dessas variáveis em relação aos resultados das partidas pode fornecer insights valiosos sobre os fatores que mais contribuem para o sucesso de uma equipe na Premier League.

This dataset contains information about football matches played in the Premier League, including the date, time, teams, results, goals scored, venue, and various match statistics. The dataset covers multiple seasons and includes matches played by different teams like Liverpool and Sheffield United.

Para fazer predições com base nesse dataset, você pode seguir os seguintes passos:

1. Análise dos dados: Comece analisando os dados disponíveis no dataset, verificando as variáveis presentes, sua distribuição e possíveis relações entre elas.

2. Pré-processamento dos dados: Realize o pré-processamento dos dados, tratando valores ausentes, convertendo variáveis categóricas em numéricas (por exemplo, usando codificação one-hot encoding) e escalando as features, se necessário.

3. Divisão dos dados: Divida o dataset em um conjunto de treinamento e um conjunto de teste para poder avaliar a performance do modelo.

4. Escolha do modelo: Escolha um modelo de classificação adequado para prever os resultados dos jogos de acordo com as features disponíveis. Alguns exemplos de modelos que podem ser utilizados são Regressão Logística, Árvores de Decisão, Random Forest, XGBoost, entre outros.

5. Treinamento do modelo: Treine o modelo utilizando o conjunto de treinamento e avalie sua performance utilizando métricas como acurácia, precisão, recall, F1-score, entre outras.

6. Validação do modelo: Após treinar o modelo, avalie sua performance utilizando o conjunto de teste. Verifique se o modelo é capaz de fazer predições precisas e confiáveis.

7. Ajustes e otimização: Caso necessário, faça ajustes no modelo, como otimização de hiperparâmetros, seleção de features mais relevantes ou até mesmo a escolha de um modelo diferente, para melhorar a performance das predições.

Lembrando que, para fazer predições precisas, é importante ter um bom entendimento do problema e dos dados disponíveis, além de realizar uma análise criteriosa e escolher os modelos mais adequados para o seu caso específico.

Projeto de Cientista de Dados - Análise de Resultados de Partidas de Futebol

Objetivo:
O objetivo deste projeto é analisar os resultados de partidas de futebol da Premier League com base nos dados fornecidos. Será realizado um estudo exploratório dos resultados e de outras variáveis para identificar padrões e tendências relacionadas ao desempenho das equipes.

Passos do Projeto:

1. Análise Exploratória dos Dados:
    - Verificar a distribuição dos resultados das partidas (vitória, derrota, empate);
    - Analisar a distribuição das variáveis numéricas, como gols feitos (GF), gols sofridos (GA), chutes (Sh), entre outras;
    - Identificar possíveis correlações entre as variáveis.

2. Análise Temporal:
    - Analisar a evolução do desempenho das equipes ao longo das temporadas;
    - Verificar se existem padrões sazonais nos resultados das partidas.
    
3. Análise de Desempenho por Equipe:
    - Analisar o desempenho de cada equipe com base nos resultados das partidas;
    - Comparar o desempenho das equipes em casa e fora de casa.

4. Modelagem Preditiva:
    - Desenvolver modelos de machine learning para prever os resultados das partidas com base nas variáveis disponíveis;
    - Avaliar a acurácia dos modelos e identificar os fatores que mais influenciam nos resultados.
    
5. Conclusões e Recomendações:
    - Apresentar as principais conclusões obtidas a partir da análise dos dados;
    - Fornecer recomendações para as equipes com base nos insights obtidos.
    
Espera-se que este projeto forneça insights valiosos sobre o desempenho das equipes da Premier League e auxilie na tomada de decisões estratégicas.

Para realizar uma análise detalhada sobre quais variáveis influenciam mais nos resultados das partidas, podemos utilizar técnicas de análise estatística descritiva, correlação e até mesmo modelos de machine learning para identificar padrões e relações entre as variáveis. Alguns passos que podem ser seguidos para essa análise incluem:

1. Análise Descritiva: Inicialmente, é importante realizar uma análise descritiva das variáveis presentes no dataset, identificando estatísticas como média, desvio padrão, distribuição e outras informações relevantes.

2. Análise de Correlação: Utilize técnicas estatísticas para verificar a correlação entre as variáveis presentes no dataset e a variável resposta 'Result'. Isso pode ajudar a identificar quais variáveis estão mais fortemente relacionadas com os resultados das partidas.

3. Feature Engineering: Crie novas variáveis que possam ser relevantes para a análise, como por exemplo, taxa de conversão de chutes a gol, posse de bola versus número de gols marcados, entre outros.

4. Modelagem: Utilize algoritmos de machine learning, como regressão logística, árvores de decisão, ou algoritmos mais avançados como Random Forest, para criar um modelo que possa prever os resultados das partidas com base nas variáveis disponíveis.

5. Avaliação do Modelo: Avalie a performance do modelo criado, utilizando métricas como acurácia, precisão, recall, entre outras, para verificar a eficácia do modelo em prever os resultados das partidas.

Esses são alguns passos básicos que podem ser seguidos para realizar uma análise mais aprofundada sobre quais variáveis estão mais relacionadas com os resultados das partidas da Premier League da temporada 2023/2024.